In [3]:
import numpy as np
import pandas as pd
import GPy
from data_preparation import *
from interpolation import *

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


### data preparation

In [52]:
sampling_distance_x = 1
sampling_distance_y = 12
length = 100 # starting with 1m
whole_map = pd.read_csv('WholeMap_Rounds_40_to_17.csv')
map = stack_map(whole_map) # create a stacked map dataframe with columns x, y, z
map = cut_map_len(map,100,length) # cut the map to the length of the map frame : 1m to 2m
known_points, unknown_points = resample(map, sampling_distance_x, sampling_distance_y) # resample the map


In [32]:
x = map.x
y = map.y
#x = map['x'].unique()#/known_points.x.max()
#y = map['y'].unique()#/known_points.y.max()

In [33]:
s1,s2 = np.meshgrid(x,y)
s = np.vstack((s1.flatten(),s2.flatten())).T

In [44]:
train_idx=known_points.index 
test_idx = unknown_points.index

In [45]:
s[train_idx].shape

(1430, 2)

In [46]:
target = known_points.z.to_numpy()[:,None]#.reshape(known_points.z.to_numpy().shape[0], 1)

In [47]:
target.shape

(1430, 1)

### testing gpy matern 32

In [48]:
#ker = GPy.kern.Exponential(2,1,1)
ker = GPy.kern.Matern32(2,1,1)
#ker = GPy.kern.RBF(2,1,1)
# create simple GP model
m = GPy.models.GPRegression(s[train_idx],target,ker)
# optimize and plot
m.optimize(messages=True)

In [49]:
z_gp_test,gp_var=m.predict(s[test_idx])
z_gp_train,gp_train=m.predict(s[train_idx])

In [60]:
mse_gpy = np.mean((z_gp_test-unknown_points.z.to_numpy())**2)

### comparisson with skg library

In [56]:
skg_matrix, skg_stacked = kriging_skg(known_points, unknown_points, 10, test_max_points=False, verbose=False)

In [57]:
mse_skg = np.mean((skg_stacked.z.to_numpy()-unknown_points.z.to_numpy())**2)

In [61]:
print(f'MSE skg: {mse_skg}')
print(f'MSE gpy: {mse_gpy}')

MSE skg: 1.3387310525701284
MSE gpy: 24.866837382508873
